# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

## Code

In [48]:
import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [2]:
outage_df = pd.read_excel('outage.xlsx')
#CLEANING
a = dict(zip(list(pd.DataFrame(outage_df.iloc[5:, 1:]).columns), outage_df.iloc[4].tolist()[1:]))
outage_cleaned = pd.DataFrame(outage_df.iloc[6:, 1:]).rename(columns=a).reset_index().drop(['index'], axis='columns')
outage_cleaned['OUTAGE.START'] = (outage_cleaned['OUTAGE.START.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.START.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned['OUTAGE.RESTORATION'] = (outage_cleaned['OUTAGE.RESTORATION.DATE'].transform(lambda x: str(x).split(' ')[0]) + ' ' +  outage_cleaned['OUTAGE.RESTORATION.TIME'].apply(str)).apply(lambda x: np.NAN if 'nan' in x else pd.to_datetime(x).to_pydatetime())
outage_cleaned = outage_cleaned.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis='columns')

In [51]:
outage_cleaned['CAUSE.CATEGORY'].value_counts().index

Index(['severe weather', 'intentional attack', 'system operability disruption',
       'public appeal', 'equipment failure', 'fuel supply emergency',
       'islanding'],
      dtype='object')

In [4]:
from sklearn.impute import SimpleImputer, KNNImputer

In [5]:
# woah = outage_cleaned[outage_cleaned['CLIMATE.CATEGORY'].notna() & outage_cleaned['TOTAL.PRICE'].notna()]
woah = outage_cleaned.copy()
woah = woah.dropna(subset=['CLIMATE.CATEGORY', 'TOTAL.PRICE', 'POPPCT_URBAN', 'OUTAGE.DURATION'])
woah['MONTH'] = woah['MONTH'].transform(str)

In [6]:
prepoc = ColumnTransformer([
    # ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
    ('climate', OneHotEncoder(handle_unknown='ignore'), ['CLIMATE.CATEGORY']),
    ('test', FunctionTransformer(lambda x: x), ['TOTAL.PRICE']),
], remainder='drop')

imputer = ColumnTransformer([
    ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
], remainder='passthrough')

pl = Pipeline([
    ('impute', imputer),
    ('prepoc', prepoc),
    ('cook', RandomForestClassifier())
])

X_train, X_test, y_train, y_test = train_test_split(woah.drop('MONTH', axis=1), woah['MONTH'], test_size=0.2)

pl.fit(X_train, y_train)

ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [ ]:
pl.score(X_test, y_test)

0.3003412969283277

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'cook__n_estimators': [50, 75, 100, 125, 150, 200, 250, 300], 
    'cook__max_features': [1, 3, 5, 7, 9, 12, 15, 18, 20], 
    'cook__min_samples_split': [2, 5, 8, 10, 12, 15]
}

grid_search = GridSearchCV(estimator=pl, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)

/Users/walterwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1680 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1680 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/walterwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/walterwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/walterwong/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklear

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('prepoc',
                                        ColumnTransformer(transformers=[('climate',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['CLIMATE.CATEGORY']),
                                                                        ('test',
                                                                         FunctionTransformer(func=<function <lambda> at 0x2916c5550>),
                                                                         ['TOTAL.PRICE'])])),
                                       ('cook', RandomForestClassifier())]),
             param_grid={'cook__max_features': [1, 3, 5, 7, 9, 12, 15, 18, 20],
                         'cook__min_samples_split': [2, 5, 8, 10, 12, 15],
                         'cook__n_estimators': [50, 75, 100, 125, 150, 200, 250,
               

In [ ]:
grid_search.best_params_

{'cook__max_features': 1,
 'cook__min_samples_split': 2,
 'cook__n_estimators': 50}

In [ ]:
pl.score(X_train, y_train)

0.9829205807002562

In [ ]:
pl.score(X_test, y_test)

0.49829351535836175

In [ ]:
woah['TOTAL.PRICE'].isna().sum()

13

In [ ]:
# woah = outage_cleaned[outage_cleaned['CLIMATE.CATEGORY'].notna() & outage_cleaned['TOTAL.PRICE'].notna()]
woah = outage_cleaned.copy()
woah = woah.dropna(subset=['CLIMATE.CATEGORY', 'TOTAL.PRICE', 'POPPCT_URBAN', 'OUTAGE.DURATION', 'RES.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS'])
woah['MONTH'] = woah['MONTH'].transform(str)
# woah['CUSTOMERS.AFFECTED'] = woah['CUSTOMERS.AFFECTED'] + 1

In [ ]:
column_to_impute = 'CUSTOMERS.AFFECTED'

# Create a KNNImputer object
imputer = KNNImputer(n_neighbors=5)

# Impute missing values in the specified column
imputed_data = pd.DataFrame(imputer.fit_transform(woah[[column_to_impute]]))

# Update the original DataFrame with the imputed values
woah.loc[:, column_to_impute] = imputed_data.values.ravel()

In [ ]:
prepoc = ColumnTransformer([
    # ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
    ('climate', OneHotEncoder(handle_unknown='ignore'), ['CLIMATE.CATEGORY']),
    # ('log', FunctionTransformer(lambda x: np.log(x + 1)), ['CUSTOMERS.AFFECTED']),
    ('test', FunctionTransformer(lambda x: x), ['TOTAL.PRICE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'RES.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS']),
], remainder='drop')

# imputer = ColumnTransformer([
#     ('imputation', SimpleImputer(strategy='median'), ['CUSTOMERS.AFFECTED']),
# ], remainder='passthrough')

pl = Pipeline([
    # ('impute', imputer),
    ('prepoc', prepoc),
    ('cook', RandomForestClassifier(max_depth=10))
])

X_train, X_test, y_train, y_test = train_test_split(woah.drop('U.S._STATE', axis=1), woah['U.S._STATE'], test_size=0.2)

pl.fit(X_train, y_train)

Pipeline(steps=[('prepoc',
                 ColumnTransformer(transformers=[('climate',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['CLIMATE.CATEGORY']),
                                                 ('test',
                                                  FunctionTransformer(func=<function <lambda> at 0x2916db160>),
                                                  ['TOTAL.PRICE',
                                                   'CUSTOMERS.AFFECTED',
                                                   'OUTAGE.DURATION',
                                                   'RES.PRICE', 'TOTAL.SALES',
                                                   'TOTAL.CUSTOMERS'])])),
                ('cook', RandomForestClassifier(max_depth=10))])

In [ ]:
pl.score(X_test, y_test)

0.8873720136518771

In [ ]:
pl.score(X_train, y_train)

0.9888983774551665

In [302]:
from sklearn.preprocessing import StandardScaler

In [620]:
# woah = outage_cleaned[outage_cleaned['CLIMATE.CATEGORY'].notna() & outage_cleaned['TOTAL.PRICE'].notna()]
woah = outage_cleaned.copy()
woah = woah.dropna(subset=['CLIMATE.CATEGORY', 'TOTAL.PRICE', 'POPPCT_URBAN', 'OUTAGE.DURATION', 'RES.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS'])
woah['MONTH'] = woah['MONTH'].transform(str)
# woah['CUSTOMERS.AFFECTED'] = woah['CUSTOMERS.AFFECTED'] + 1
column_to_impute = 'CUSTOMERS.AFFECTED'

# Create a KNNImputer object
imputer = KNNImputer(n_neighbors=5)

# Impute missing values in the specified column
imputed_data = pd.DataFrame(imputer.fit_transform(woah[[column_to_impute]]))

# Update the original DataFrame with the imputed values
woah.loc[:, column_to_impute] = imputed_data.values.ravel()
prepoc = ColumnTransformer([
    ('climate', OneHotEncoder(handle_unknown='ignore'), ['CLIMATE.CATEGORY']), #comment out for baseline
    ('log', FunctionTransformer(lambda x: np.log(x + 1)), ['CUSTOMERS.AFFECTED']), # comment out for baseline
    ('square', FunctionTransformer(lambda x: x**2), ['RES.PRICE']),
    ('test', FunctionTransformer(lambda x: x), ['TOTAL.PRICE', 'CUSTOMERS.AFFECTED', 'OUTAGE.DURATION', 'RES.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS']),
    # ('test', FunctionTransformer(lambda x: x), ['TOTAL.PRICE', 'OUTAGE.DURATION']), #baseline
], remainder='drop')


pl = Pipeline([
    ('prepoc', prepoc),
    ('cook', RandomForestClassifier())
])

X_train, X_test, y_train, y_test = train_test_split(woah.drop('CAUSE.CATEGORY', axis=1), woah['CAUSE.CATEGORY'], test_size=0.2)

pl.fit(X_train, y_train)

Pipeline(steps=[('prepoc',
                 ColumnTransformer(transformers=[('climate',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['CLIMATE.CATEGORY']),
                                                 ('log',
                                                  FunctionTransformer(func=<function <lambda> at 0x29b426c10>),
                                                  ['CUSTOMERS.AFFECTED']),
                                                 ('std', StandardScaler(),
                                                  ['OUTAGE.DURATION']),
                                                 ('square',
                                                  FunctionTransformer(func=<function <lambda> at 0x29b426e50>),
                                                  ['RES.PRICE']),
                                                 ('test',
                                                  FunctionTran

In [621]:
pl.score(X_test, y_test)

0.7508532423208191

In [622]:
pl.score(X_train, y_train)

1.0

### Framing the Problem

Power outages are a large concern to functioning societies, with massive implications for daily life, commerce, and infrastructure. However, the underlying cause of power outages can vary from natural disasters to equipment failure. To be cognizant of the power outage's cause can provide insight to appropriately respond to each case. For instance, when a power outage results from a xxx, the impact can be devastating with prolonged after effects. On the other hand, outages caused by human error or technical issues often represent a minor inconvenience. 

By understanding the underlying cause of a power outage, communities can better prepare and respond appropriately, whether it involves implementing robust disaster recovery plans for natural disasters or focusing on regular maintenance and upgrades to mitigate the risk of equipment failures. This awareness enables a more targeted and efficient approach to minimize the impact of power outages on society at large.

We will utilize features:


We aim to train a classification model using this data that can assist in identifying the CAUSE.CATEGORY of the power outage.....

these feature will be preent before identification of the cause....


## Data Cleaning
In the original data in the excel file, the 5 rows are unwanted as they are just the title and some notes about the data. The dataframe and its columns "start" from the 6th row downward. Additionally, below the columns, there is a row to specify the units of the column if necessary. We also removed this columns when cleaning the data as there are no actual observations.

At this point, the data being loaded has all the columns and observations in the excel file data. However, the columns OUTAGE.START.DATE and OUTAGE.START.TIME can be combined as the first always has a time of "00:00:00". The same is true for the restoration date and time. We combined these columns into OUTAGE.START and OUTAGE.RESTORATION, dropping the original 4.

## Response Variable
The response variable will be the CAUSE.CATEGORY which has these options:
- 'severe weather'
- 'intentional attack'
- 'system operability disruption'
- 'public appeal', 
- 'equipment failure', 
- 'fuel supply emergency'
- 'islanding'"

## Justification
blah blah blah


## Features
To train the model, we will use ...... These columns will provide relevant information for predicting the power outage’s location. At the time of prediction, we would have access to the features mentioned above for the power outage in question. We would not have any future information beyond what is available in the dataset.



## Metric for Evaluation
we will use chi-squared

## Justification for Metric

### Baseline Model

In [ ]:
# TODO

### Final Model

In [ ]:
# TODO

### Fairness Analysis

In [ ]:
# TODO